## Setup

In [ ]:
import pandas as pd
import numpy as np
import yaml
import openai
import asyncio 
import shap
import pickle
from autogen_core.models import AssistantMessage, FunctionExecutionResult, FunctionExecutionResultMessage, UserMessage
from autogen_agentchat.agents import AssistantAgent, CodeExecutorAgent
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import TextMentionTermination, MaxMessageTermination
from autogen_agentchat.ui import Console
from autogen_ext.code_executors.local import LocalCommandLineCodeExecutor
from autogen_agentchat.messages import TextMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_core import CancellationToken
import sys
import os


sys.path.append("../SHAPnarrative-metrics")
sys.path.append("../script")

from shapnarrative_metrics.llm_tools import llm_wrappers
from shapnarrative_metrics.misc_tools.manipulations import full_inversion, shap_permutation
from shapnarrative_metrics.llm_tools.generation import GenerationModel
from shapnarrative_metrics.llm_tools.extraction import ExtractionModel
from FaithfulEvaluator import FaithfulEvaluatorAgent
from Narrator import NarratorAgent


c:\Users\yhe\Desktop\PHD\Research\25spring_Agentic_AI_project\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### API Key & dataset

In [2]:
with open("../config/keys.yaml") as f:
    dict=yaml.safe_load(f)
api_key = dict["API_keys"]["OpenAI"]

dataset_name="student"

with open(f'../data/{dataset_name}_dataset/dataset_info', 'rb') as f:
   ds_info= pickle.load(f)

with open(f'../data/{dataset_name}_dataset/RF.pkl', 'rb') as f:
   trained_model=pickle.load(f)

train=pd.read_parquet(f"../data/{dataset_name}_dataset/train_cleaned.parquet")
test=pd.read_parquet(f"../data/{dataset_name}_dataset/test_cleaned.parquet")

print(test.index)

Index([252, 236, 275, 148, 309,  33, 248, 150, 346, 246, 221, 261,   9, 204,
       329, 207, 332,  59, 371,  11, 300, 299, 159, 280, 175, 374, 196, 189,
       188, 211, 307, 284,  36,  72, 114, 274,  82, 345,  54, 385,  26, 190,
       120, 212, 170,  24,  13,  95, 237, 285, 264,  20, 181, 107, 171, 223,
       192,  42,  48,  15, 258, 228,  41,  31,  75, 102, 173, 373, 147,  79,
       293,  91,  52, 179, 200, 199, 320, 375, 245],
      dtype='int64')


#### Choose Data instance & Generate SHAP table, prompt and story

In [6]:
idx=171

x=test[test.columns[0:-1]].loc[[idx]]
y=test[test.columns[-1]].loc[[idx]]

TEMPERATURE=0
MANIP=False

gpt = llm_wrappers.GptApi(api_key, model="gpt-4o", system_role="You are a teacher that explains AI predictions.", temperature=TEMPERATURE)
generator=GenerationModel(ds_info=ds_info, llm=gpt)
generator.gen_variables(trained_model,x,y,tree=True)
#shap_df here is always the one without any manipulation
shap_df=pd.DataFrame(generator.explanation_list[0].head(4))
prompt = generator.generate_story_prompt(0,prompt_type="long",manipulate=MANIP)
print(shap_df)

  feature_name  SHAP_value  feature_value  feature_average  \
0         Fedu   -0.038863              0         2.515823   
1   Mjob_other   -0.038738              1         0.370253   
2     failures    0.035703              0         0.360759   
3     romantic   -0.033848              1         0.322785   

                                        feature_desc  
0  Father's education (0: none, 1: primary educat...  
1        One-hot variable for mothers's job -- other  
2        Number of past class failures (from 0 to 3)  
3  Student is in a romantic relationship (0:no, 1...  


Load the defined system messages

In [7]:
from system_messages import (
    system_message_narrator,
    system_message_narrator_MANIP,
    system_message_faithful_critic,
    system_message_faithful_critic_MANIP,
    system_message_coherence,
)

if MANIP:
    narrator_sys_msg = system_message_narrator_MANIP
    faithful_critic_sys_msg = system_message_faithful_critic_MANIP
    coherence_sys_msg=system_message_coherence
    print ("The SHAP table is manipulated")
else:
    narrator_sys_msg = system_message_narrator
    faithful_critic_sys_msg = system_message_faithful_critic
    coherence_sys_msg=system_message_coherence
    print ("The SHAP table is normal (not manipulated)")

The SHAP table is normal (not manipulated)


 #### <span style="color: yellow ">First step:  coherence agent no function call
   
<span style="color: yellow ">1. coherenceagent only use plain language system message, without any functioncall or perplexity.
<span style="color: yellow ">2. after reaching 100% faithulness, the coherenceagent start to work.

In [ ]:
# from autogen_agentchat.teams import SelectorGroupChat
# from autogen_agentchat.messages import AgentEvent, ChatMessage
# from selector_func import selector_func
# import re

# shap_df = shap_df

# async def main() -> None:
#     SEED = 42
#     TEMPERATURE = 0

#     # Initialize OpenAI client
#     model_client = OpenAIChatCompletionClient(
#         model="gpt-4o",
#         api_key=api_key,
#         seed=SEED,
#         temperature=TEMPERATURE
#     )
    
#     # Create the narrator agent
#     narrator = NarratorAgent(
#         name="narrator",
#         system_message=narrator_sys_msg, 
#         model_client=model_client,
#         reflect_on_tool_use=False,
#     )
    
#     # Create the critic agent with advanced extraction capabilities
#     faithful_evaluator = FaithfulEvaluatorAgent(
#         name="faithful_evaluator",
#     )
    
#     # Set up the models for the critic - THIS IS IMPORTANT
#     faithful_evaluator.set_models(
#         extractor_class=ExtractionModel,
#         generator_class=GenerationModel,
#         ds_info=ds_info,
#         llm=gpt
#     )

#     faithful_evaluator.default_shap_df = shap_df 

#     faithful_critic = AssistantAgent(
#         name="faithful_critic",
#         system_message=faithful_critic_sys_msg,
#         model_client=model_client,
#         reflect_on_tool_use=False,
#     )

#     coherenceagent = AssistantAgent(
#             name="coherenceagent",
#             system_message=coherence_sys_msg, 
#             model_client=model_client,
#             reflect_on_tool_use=False,
#         )

#     # Create the group chat with custom processing
#     termination = TextMentionTermination("TERMINATE") | MaxMessageTermination(20)
    
#     group_chat = SelectorGroupChat(
#         [narrator, faithful_evaluator, faithful_critic, coherenceagent],
#         model_client=model_client,
#         selector_func=selector_func,
#         termination_condition=termination,
#     )

#     # Run the chat
#     await Console(group_chat.run_stream(task=prompt))

# if __name__ == "__main__":
#     await main()

---------- user ----------


        Your goal is to generate a textual explanation or narrative as to why an AI model made a certain prediction for one particular instance. 
        To do this, you will be provided with a dictionary that contains broad descriptions of the specific dataset, target variable, and the task the model was trained on.
        Additionally, you will be provided with a dataframe that contains the names of all the features, their descriptions, their values, their average values and their SHAP values.
        Finally you will get a single string describing the result of the prediction.

        The goal of SHAP is to explain the prediction of an instance by computing the contribution of each feature to the prediction.
        Each individual SHAP value is a measure of how much additional probability this feature adds or subtracts 
        in the predicted probability relative to the base level probability. 
        This relative nature of the SHAP values might h

In [ ]:
import aiohttp
import asyncio
import json
from autogen_agentchat.agents import ConversableAgent

# The URL from ngrok that you'll get from running the Colab notebook
PERPLEXITY_API_URL = "https://xxxx-xx-xx-xxx-xxx.ngrok.io/perplexity"

async def get_perplexity(text):
    """Async function to get perplexity from our API"""
    async with aiohttp.ClientSession() as session:
        async with session.post(
            PERPLEXITY_API_URL,
            json={"text": text}
        ) as response:
            if response.status == 200:
                result = await response.json()
                return result["perplexity"]
            else:
                error = await response.text()
                raise Exception(f"API Error: {error}")

# Define your CoherenceAgent that uses the perplexity API
class CoherenceAgent(ConversableAgent):
    async def analyze_coherence(self, narrative):
        """Analyze the coherence of a narrative using perplexity"""
        try:
            perplexity_score = await get_perplexity(narrative)
            
            # Create a coherence analysis based on perplexity
            analysis = f"Perplexity analysis: {perplexity_score:.2f}\n\n"
            
            if perplexity_score < 50:
                analysis += "The narrative has excellent coherence! The flow is natural and easy to follow."
            elif perplexity_score < 100:
                analysis += "The narrative has good coherence, but some minor improvements could be made."
            elif perplexity_score < 200:
                analysis += "The narrative shows moderate coherence issues. Consider restructuring some sentences."
            else:
                analysis += "The narrative has significant coherence problems. Consider simplifying complex parts."
                
            return analysis
        except Exception as e:
            return f"Error calculating perplexity: {str(e)}"
    
    async def _process_message(self, message):
        if "analyze_coherence" in message.content:
            # Extract narrative text from message
            # This is simplified - you'll need to parse the actual message
            narrative = message.content.replace("analyze_coherence:", "").strip()
            analysis = await self.analyze_coherence(narrative)
            return analysis
        return super()._process_message(message)

# Define your agent system
def setup_agents():
    narrator = autogen.ConversableAgent(
        name="narrator",
        system_message="You are a narrator who creates clear narratives."
    )
    
    faithful_evaluator = autogen.ConversableAgent(
        name="faithful_evaluator",
        system_message="You evaluate if narratives are faithful to data."
    )
    
    faithful_critic = autogen.ConversableAgent(
        name="faithful_critic",
        system_message="You critique narratives for faithfulness."
    )
    
    coherence_agent = CoherenceAgent(
        name="coherence_agent",
        system_message="You analyze the coherence of narratives."
    )
    
    # Set up your groupchat with the agents
    # ... existing setup code ...
    
    return narrator, faithful_evaluator, faithful_critic, coherence_agent

# In your main function
async def main():
    # Set up agents
    narrator, faithful_evaluator, faithful_critic, coherence_agent = setup_agents()
    
    # Test the perplexity calculation
    test_narrative = "This is a test narrative. It should be coherent and well-structured."
    analysis = await coherence_agent.analyze_coherence(test_narrative)
    print(analysis)
    
    # Rest of your agent interaction code
    # ...

if __name__ == "__main__":
    asyncio.run(main())